In [2]:
# =============================================================================
# Import Libraries
# =============================================================================
import requests                     # For making API calls to CoinMarketCap
import pandas as pd                 # For data manipulation and analysis
import numpy as np                  # For numerical operations
import matplotlib.pyplot as plt     # For plotting graphs
import datetime                     # For handling date and time information
import tweepy                       # For accessing the Twitter API
import nltk                         # For Natural Language Processing
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # VADER for sentiment analysis
from sklearn.linear_model import LinearRegression           # For predictive modeling
from sklearn.metrics import mean_absolute_error, r2_score      # For model evaluation